In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import pymysql
import re
from dotenv import load_dotenv
load_dotenv()
import os

In [2]:
db_connection_str = os.getenv("DB_CONNECTION_STR")

db_connection = create_engine(db_connection_str)

df = pd.read_sql('SELECT * FROM crash_db_Denver', con=db_connection)
df = df.replace('',np.nan)

In [3]:
df.shape

(173106, 49)

In [4]:
df.head()

,cid,createDate,createBy,dataFrom,OBJECTID_1,INCIDENT_ID,OFFENSE_ID,OFFENSE_CODE,OFFENSE_CODE_EXTENSION,TOP_TRAFFIC_ACCIDENT_OFFENSE,...,TU2_VEHICLE_MOVEMENT,TU2_DRIVER_ACTION,TU2_DRIVER_HUMANCONTRIBFACTOR,TU2_PEDESTRIAN_ACTION,SERIOUSLY_INJURED,FATALITIES,FATALITY_MODE_1,FATALITY_MODE_2,SERIOUSLY_INJURED_MODE_1,SERIOUSLY_INJURED_MODE_2
0,1000000001,2020-01-28 05:26:38,David,Denver,OBJECTID_1,INCIDENT_ID,OFFENSE_ID,OFFENSE_CODE,OFFENSE_CODE_EXTENSION,TOP_TRAFFIC_ACCIDENT_OFFENSE,...,TU2_VEHICLE_MOVEMENT,TU2_DRIVER_ACTION,TU2_DRIVER_HUMANCONTRIBFACTOR,TU2_PEDESTRIAN_ACTION,SERIOUSLY_INJURED,FATALITIES,FATALITY_MODE_1,FATALITY_MODE_2,SERIOUSLY_INJURED_MODE_1,SERIOUSLY_INJURED_MODE_2\r
1,1000000002,2020-01-28 05:26:38,David,Denver,1,20193963.0,2019396354010,5401,0,TRAF - ACCIDENT - HIT & RUN,...,GOING STRAIGHT,OTHER,NO APPARENT,,0.0,0.0,,,,\r
2,1000000003,2020-01-28 05:26:38,David,Denver,2,20193966.0,2019396654410,5441,0,TRAF - ACCIDENT,...,GOING STRAIGHT,OTHER,NO APPARENT,,0.0,0.0,,,,\r
3,1000000004,2020-01-28 05:26:38,David,Denver,3,20193991.0,2019399154010,5401,0,TRAF - ACCIDENT - HIT & RUN,...,PARKED,,,,0.0,0.0,,,,\r
4,1000000005,2020-01-28 05:26:38,David,Denver,4,20194077.0,2019407754410,5441,0,TRAF - ACCIDENT,...,GOING STRAIGHT,OTHER,NO APPARENT,,0.0,0.0,,,,\r


In [5]:
df = df[1:]

In [6]:
df.columns

Index(['cid', 'createDate', 'createBy', 'dataFrom', 'OBJECTID_1',
       'INCIDENT_ID', 'OFFENSE_ID', 'OFFENSE_CODE', 'OFFENSE_CODE_EXTENSION',
       'TOP_TRAFFIC_ACCIDENT_OFFENSE', 'FIRST_OCCURRENCE_DATE',
       'LAST_OCCURRENCE_DATE', 'REPORTED_DATE', 'INCIDENT_ADDRESS', 'GEO_X',
       'GEO_Y', 'GEO_LON', 'GEO_LAT', 'DISTRICT_ID', 'PRECINCT_ID',
       'NEIGHBORHOOD_ID', 'BICYCLE_IND', 'PEDESTRIAN_IND',
       'HARMFUL_EVENT_SEQ_1', 'HARMFUL_EVENT_SEQ_2', 'HARMFUL_EVENT_SEQ_3',
       'ROAD_LOCATION', 'ROAD_DESCRIPTION', 'ROAD_CONTOUR', 'ROAD_CONDITION',
       'LIGHT_CONDITION', 'TU1_VEHICLE_TYPE', 'TU1_TRAVEL_DIRECTION',
       'TU1_VEHICLE_MOVEMENT', 'TU1_DRIVER_ACTION',
       'TU1_DRIVER_HUMANCONTRIBFACTOR', 'TU1_PEDESTRIAN_ACTION',
       'TU2_VEHICLE_TYPE', 'TU2_TRAVEL_DIRECTION', 'TU2_VEHICLE_MOVEMENT',
       'TU2_DRIVER_ACTION', 'TU2_DRIVER_HUMANCONTRIBFACTOR',
       'TU2_PEDESTRIAN_ACTION', 'SERIOUSLY_INJURED', 'FATALITIES',
       'FATALITY_MODE_1', 'FATALITY_MODE_2',

In [7]:
df2 = pd.DataFrame()
df2['cid'] = df.cid
df2['city'] = 'Denver'
df2['crash_time'] = df.REPORTED_DATE
df2['latitude'] = df.GEO_LAT
df2['longitude'] = df.GEO_LON
df.SERIOUSLY_INJURED = df.SERIOUSLY_INJURED.fillna(0)
df.SERIOUSLY_INJURED = df.SERIOUSLY_INJURED.astype('float').astype('int')
df.FATALITIES = df.FATALITIES.fillna(0)
df.FATALITIES = df.FATALITIES.astype('float').astype('int')
df2['injured_count'] = df.SERIOUSLY_INJURED + df.FATALITIES
df2['street_name'] = df.INCIDENT_ADDRESS

df2['crash_time'] = pd.to_datetime(df2.crash_time)
df2['crash_date'] = pd.to_datetime(df2['crash_time'].dt.date)

In [8]:
df2.head()

,cid,city,crash_time,latitude,longitude,injured_count,street_name,crash_date
1,1000000002,Denver,2019-01-02 21:01:00,39.7404758631,-105.025220472,0,W COLFAX AVE / N FEDERAL BLVD,2019-01-02
2,1000000003,Denver,2019-01-02 20:59:00,39.7289369845,-104.992859113,0,N DELAWARE ST / W 8TH AVE,2019-01-02
3,1000000004,Denver,2019-01-02 21:14:00,39.7636141924,-104.929109267,0,5000 E 33RD AVE,2019-01-02
4,1000000005,Denver,2019-01-02 22:46:00,39.7840735673,-104.991179215,0,W 48TH AVE / N BANNOCK ST,2019-01-02
5,1000000006,Denver,2019-01-02 23:34:00,39.7832654948,-105.006428226,0,I70 HWYWB / N PECOS ST,2019-01-02


In [9]:
df2.to_sql('cleaned_crash_Denver',con = db_connection, if_exists="replace",index=False)

In [10]:
df2.to_sql('home_allcleanedcrashdata',con = db_connection, if_exists="append",index=False)